In [65]:
import torch
from models import MVCNN
from torchvision import transforms
import os
from PIL import Image
import re
import random 

In [66]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [67]:
svcnn = MVCNN.SVCNN(name="VGG11_SVCNN")
svcnn_weights = torch.load(
    "trained-models/SVCNN/model-00010.pth", weights_only=True, map_location=device
)
svcnn.load_state_dict(svcnn_weights, strict=False)
svcnn.eval()
svcnn.to(device)

mvcnn = MVCNN.MVCNN(name="VGG11_MVCNN", model=svcnn)
mvcnn_weights = torch.load(
    "trained-models/MVCNN/model-00008.pth", weights_only=True, map_location=device
)
mvcnn.load_state_dict(mvcnn_weights, strict=False)
mvcnn.eval()
mvcnn.to(device)

MVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [68]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

In [69]:
def init_csv():
    with open("svcnn-grads.csv", "w") as file:
        file.write("model,model_id,imp1,imp2,imp3,imp4,imp5,imp6,imp7,imp8,imp9,imp10,imp11,imp12\n")
    print("File Initialised")

In [70]:
def save_to_csv(grads: dict, model: str, model_id: int):
    with open ('svcnn-grads.csv', 'a') as file:
        file.write(f"{model},{model_id},")
        for i in range(0, 12):
            file.write(f"{grads[i]},")
        file.write("\n")
    print("File Updated")

In [71]:
def run_svcnn(svcnn, images_dir):
    outputs = []
    images = os.listdir(images_dir)
    images = sorted(
        images, key=lambda x: int(re.search(r'_(\d+)\.png', x)[1])
    )
    # images = [f"{images_dir}/{image}" for image in images]
    images = [os.path.join(images_dir, image) for image in images]
    svcnn = svcnn.to(device)
    svcnn.eval()
    # print(images)
    for _ in range(12):
        image = Image.open(images[_])
        image = image.convert('RGB')
        image = transforms(image)
        image = image.to(device)
        image = image.unsqueeze(0)
        image = image.to(device)
        with torch.no_grad():
            output = svcnn(image)
        outputs.append(output)
    pooled_features, _ = torch.max(torch.stack(outputs), dim=0, keepdim=False)
    pooled_features = pooled_features.view(1, -1)
    pooled_features = pooled_features.to(device)
    # print(pooled_features.shape)
    return pooled_features

In [72]:
path ='./ModelNet40-12View/'
objects = [obj for obj in os.listdir(path) if not obj.startswith('.')] 
object_paths = [os.path.join(path, i, 'test') for i in objects]

model_paths = []
for obj_path in object_paths:
    if not os.path.isdir(obj_path):
        continue
    models_in_folder = [
        os.path.join(obj_path, fname)
        for fname in os.listdir(obj_path)
        if not fname.startswith('.')  # Exclude .DS_Store or other hidden files
    ]
    model_paths.extend(models_in_folder)

In [73]:
def rem_dup(arr):
    done = []
    for i in arr:
        if i not in done:
            done.append(i)
    return done

In [74]:
models = []
x = 0
for i in range(0, len(model_paths)):
    model = model_paths[i].split('/')[-3]
    models.append(model)
    
all_models = {}
models.sort()
models = rem_dup(models)
max_len = max(len(model) for model in models)
print('*'*53,"Models",'*'*53)
for i in range(0, len(models)):
    print(f"Model: {models[i].ljust(max_len)}\tClass: {i}",end='\t')
    all_models[models[i]] = i
    if i % 3 == 0:
        print()

***************************************************** Models *****************************************************
Model: airplane  	Class: 0	
Model: bed       	Class: 1	Model: bench     	Class: 2	Model: bookshelf 	Class: 3	
Model: bottle    	Class: 4	Model: bowl      	Class: 5	Model: car       	Class: 6	
Model: chair     	Class: 7	Model: cone      	Class: 8	Model: cup       	Class: 9	
Model: curtain   	Class: 10	Model: door      	Class: 11	Model: flower_pot	Class: 12	
Model: glass_box 	Class: 13	Model: guitar    	Class: 14	Model: keyboard  	Class: 15	
Model: lamp      	Class: 16	Model: laptop    	Class: 17	Model: mantel    	Class: 18	
Model: person    	Class: 19	Model: piano     	Class: 20	Model: plant     	Class: 21	
Model: radio     	Class: 22	Model: range_hood	Class: 23	Model: sink      	Class: 24	
Model: stairs    	Class: 25	Model: stool     	Class: 26	Model: tent      	Class: 27	
Model: toilet    	Class: 28	Model: tv_stand  	Class: 29	Model: vase      	Class: 30	
Model: wardrobe 

In [75]:
count = 0
random.shuffle(model_paths)
tests = 1000
correct = 0
for model_path in model_paths:
    count+=1
    pooled_features = run_svcnn(svcnn.net_1, model_path)
    with torch.no_grad():
        mvcnn = mvcnn.to(device)
        mvcnn.eval()
        pooled_features = pooled_features.to(device)
        final_out = torch.argmax(mvcnn.net_2(pooled_features), dim=1)
    model = model_path.split('/')[2]
    model_class = model_path.split('/')[4]
    print(f"Count: {count},\tModel: {model}, Model Class: {model_class}, Actual Class {all_models[model]}, Predicted Class: {final_out.item()}")
    if int(all_models[model]) == int(final_out.item()):
        correct+=1
    if count > tests:
        break
    
print(f"Accuracy: {correct}/{count} = ", (correct/count)*100, "%")

Count: 1,	Model: plant, Model Class: plant_0331, Actual Class 21, Predicted Class: 26
Count: 2,	Model: bowl, Model Class: bowl_0082, Actual Class 5, Predicted Class: 6
Count: 3,	Model: plant, Model Class: plant_0300, Actual Class 21, Predicted Class: 26
Count: 4,	Model: chair, Model Class: chair_0915, Actual Class 7, Predicted Class: 8
Count: 5,	Model: bookshelf, Model Class: bookshelf_0636, Actual Class 3, Predicted Class: 36
Count: 6,	Model: bowl, Model Class: bowl_0066, Actual Class 5, Predicted Class: 6
Count: 7,	Model: mantel, Model Class: mantel_0337, Actual Class 18, Predicted Class: 21
Count: 8,	Model: bookshelf, Model Class: bookshelf_0579, Actual Class 3, Predicted Class: 4
Count: 9,	Model: glass_box, Model Class: glass_box_0269, Actual Class 13, Predicted Class: 16
Count: 10,	Model: toilet, Model Class: toilet_0411, Actual Class 28, Predicted Class: 35
Count: 11,	Model: vase, Model Class: vase_0485, Actual Class 30, Predicted Class: 6
Count: 12,	Model: lamp, Model Class: lam

In [481]:
final_out[0, final_label].backward()

In [482]:
gradients = []
for i in range(12):
    if outputs[i].grad is not None:
        grad_norm = outputs[i].grad.norm().item()
        gradients.append(grad_norm)
    else:
        gradients.append(0.0)

IndexError: list index out of range

In [ ]:
most_influential_view = gradients.index(max(gradients))
grads = {}
print("Most influential view index:", most_influential_view)
print("Gradient norms for each view:")
for i in range(12):
    print(f"View {i:2d}: {gradients[i]:.3f}: Image: {images_12_view[i]}")
    grads[i] = gradients[i]

Most influential view index: 0
Gradient norms for each view:
View  0: 0.838: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_0.png
View  1: 0.131: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_1.png
View  2: 0.114: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_2.png
View  3: 0.095: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_3.png
View  4: 0.103: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_4.png
View  5: 0.117: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_5.png
View  6: 0.116: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_6.png
View  7: 0.122: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_7.png
View  8: 0.097: Image: ./ModelNet40-12View/bookshelf/test/bookshelf_0575/bookshelf_0575_shaded_8.png
View  9: 0.101: Image: ./Model

In [ ]:
print(grads)

{0: 0.837579607963562, 1: 0.131391704082489, 2: 0.11441924422979355, 3: 0.09518155455589294, 4: 0.10292992740869522, 5: 0.11653438955545425, 6: 0.11563097685575485, 7: 0.12186631560325623, 8: 0.09652998298406601, 9: 0.10089205950498581, 10: 0.09913935512304306, 11: 0.11590941995382309}


In [ ]:
sorted_by_values_asc = dict(sorted(grads.items(), key=lambda item: item[1], reverse=True))

for key in sorted_by_values_asc:
    print(f"View {key}: Gradient Norm: {sorted_by_values_asc[key]}")


View 0: Gradient Norm: 0.837579607963562
View 1: Gradient Norm: 0.131391704082489
View 7: Gradient Norm: 0.12186631560325623
View 5: Gradient Norm: 0.11653438955545425
View 11: Gradient Norm: 0.11590941995382309
View 6: Gradient Norm: 0.11563097685575485
View 2: Gradient Norm: 0.11441924422979355
View 4: Gradient Norm: 0.10292992740869522
View 9: Gradient Norm: 0.10089205950498581
View 10: Gradient Norm: 0.09913935512304306
View 8: Gradient Norm: 0.09652998298406601
View 3: Gradient Norm: 0.09518155455589294
